In [1]:
import csv
import os
import numpy as np
import pandas as pd
import gc
from datetime import timedelta, date
import uuid
import traceback

# import rsa_data_summary as rd
# import rsa_data_wim as wim
# import rsa_headers as rh
import config
import queries as q
import tools

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
PATH = r"C:\PQ410"

In [15]:
def get_files(path: str):
    """
    It takes a list of files, checks if they are in a zip file, if not, it gets the files from the
    directory, if they are in a zip file, it exits. 

    Then it checks if the file exists, if not, it creates it. 

    Then it reads the file, if it can't read it, it creates an empty list. 

    Then it checks if the files are in the list, if they are, it removes them. 

    Then it checks if the directory exists, if not, it creates it. 

    Then it checks if the file exists, if not, it creates it. 

    Then it returns the list of files.

    :param files: str = "C:/Users/user/Desktop/files"
    :type files: str
    :return: A list of files that are not in the fileComplete list.
    """
    if not os.path.exists(os.path.expanduser(config.PATH)):
        os.makedirs(os.path.expanduser(config.PATH))

    if not os.path.exists(os.path.expanduser(config.FILES_COMPLETE)):
        with open(
                os.path.expanduser(config.FILES_COMPLETE), "w",) as f:
            pass

    if not os.path.exists(os.path.expanduser(config.PATH)):
        with open(os.path.expanduser(config.FILES_FAILED), "w",) as f:
            pass

    file_complete = pd.read_csv(config.FILES_COMPLETE, header=None)
    file_complete = file_complete[0].tolist()

    if tools.is_zip(path) == False:
        files = tools.get_files(path)
    else:
        raise SystemExit

    files = [i for i in files if i not in file_complete]

    return files

In [16]:
files = get_files(PATH)

getting files
COLLECTING FILES......


In [3]:
    print("getting files")
    if not os.path.exists(os.path.expanduser(config.PATH)):
        os.makedirs(os.path.expanduser(config.PATH))

    if not os.path.exists(os.path.expanduser(config.FILES_COMPLETE)):
        with open(
                os.path.expanduser(config.FILES_COMPLETE), "w",) as f:
            pass

    if not os.path.exists(os.path.expanduser(config.PATH)):
        with open(os.path.expanduser(config.FILES_FAILED), "w",) as f:
            pass

    files_complete = pd.read_csv(config.FILES_COMPLETE, header=None)
    files_complete = file_complete[0].tolist()

    if tools.is_zip(path) == False:
        files = tools.get_files(path)
    else:
        raise SystemExit

    files = [i for i in files if i not in file_complete]
    print(files)

getting files


KeyError: 0

In [2]:
HEADER_IDS_FOR_UPDATE_MAIN_WITH_T30 = """SELECT header_id FROM trafc.electronic_count_data_partitioned p WHERE exists(
    SELECT id FROM trafc.electronic_count_data_type_30 t30 WHERE t30.header_id = p.header_id
    and p.light_motor_vehicles is null
    limit 1
    )"""

In [ ]:
header_ids = list(pd.read_sql(HEADER_IDS_FOR_UPDATE_MAIN_WITH_T30, config.ENGINE))

In [2]:
# individual tables for type 30 and 60 test
# files = tools.getfiles(r'C:\FTP\Syntell\SMEC RSA Files_GP PRM Sites_Dec21toFeb22')
test_header = r"C:\PQ410\2 - Data In\TIS Data\CSIR\2010\Secondary Loop\0007.rsa"
file = r"C:\PQ410\2 - Data In\TIS Data\GDRT\GP_RSA 2014\0038.RSA"
t21_file =r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_1.RSA"
t10_file = r"C:\FTP\Syntell\SMEC RSV Files_GP PRM Sites_Dec21toFeb22_individuals\0400-20220222.RSV"
t10_file2 = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"
type60_file = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"
new_test = r"C:\PQ410\2 - Data In\TIS Data\Disks Sept 2020\Data Discs\2017\2017Q2\Historic\GPG CTO 2011 Yearbook\RSA Data\0317.rsa"
another_test = r"C:\PQ410\2 - Data In\TIS Data\CSIR\2010\Permanent HSWIM\0087_2010.rsa"
empty = r

NameError: name 'r' is not defined

In [36]:
# electronic_count_data_type_21_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_21 limit 1",config.ENGINE).columns)
# electronic_count_data_type_30_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_30 limit 1",config.ENGINE).columns)
# electronic_count_data_type_60_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_60 limit 1",config.ENGINE).columns)
# electronic_count_data_type_70_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_70 limit 1",config.ENGINE).columns)
# header_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_header limit 1",config.ENGINE).columns)
vm_limits = pd.read_sql_query(
            f"SELECT * FROM {config.TRAFFIC_LOOKUP_SCHEMA}.gross_vehicle_mass_limits;"
            , config.ENGINE)
t10_cols = list(pd.read_sql_query(
            q.SELECT_ELECTRONIC_COUNT_DATA_TYPE_10_LIMIT1, config.ENGINE).columns)

In [3]:
# df = tools.to_df(t10_file)
# dfa = pd.read_csv(t10_file, header=None, nrows = 200) #, sep='\s+', engine='python')
dfa = pd.read_csv(another_test, header=None, sep='\s+\t', engine='python', nrows = 500)
dfa = dfa[0].str.split("\s+|,\s+|,|;|;\s+", expand=True)

In [235]:
df = pd.DataFrame(dfa.loc[ # data
    (~dfa[0].isin(["H0", "H9", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
    & ((
        (dfa[0].isin(["21", "22", "70", "30", "31", "13", "60"]))
        & (dfa[1].isin(["0", "1", "2", "3", "4"]))
        & (dfa.loc[dfa[0].isin(["21", "70", "30", "13", "60"]), 3].fillna("0").astype(int) > 80)
    ) | (
        (dfa[0].isin(["10"]))
        & (~dfa[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
        & (dfa.loc[dfa[0].isin(["10"]), 4].astype(int) > 80)
    ))           
    ]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [4]:
head_df = pd.DataFrame(dfa.loc[ # header
    (dfa[0].isin(["H0", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
    | (
        (dfa[0].isin(["21", "70", "30", "13", "60"]))
        & (dfa.loc[dfa[0].isin(["21", "70", "30", "13", "60"])][2].fillna("0").astype(int) < 80)
    )
    | (
        (dfa[0].isin(["10"]))
        & (dfa[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
    )
]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [6]:
df = head_df

In [27]:
# def header(self, df: pd.DataFrame) -> pd.DataFrame:
"""
It takes a dataframe, finds the header information, and returns a dataframe with the header
information

:param df: pd.DataFrame
:type df: pd.DataFrame
:return: A dataframe
"""
df = df.reset_index(drop=True)
st_nd = df.loc[df[0].isin(["D1", "D3"]), 0:4].reset_index(
    drop=True).copy()

# standard column identifiers
st_dt_col = 1
end_dt_col = 3
st_tme_col = 2
end_tme_col = 4

# checks for all non numeric characters in the date and removes them
st_nd[st_dt_col] = st_nd[st_dt_col].str.extract(
    "(\d+)", expand=False)
st_nd[end_dt_col] = st_nd[end_dt_col].str.extract(
    "(\d+)", expand=False)
st_nd[st_tme_col] = st_nd[st_tme_col].str.extract(
    "(\d+)", expand=False)
st_nd[end_tme_col] = st_nd[end_tme_col].str.extract(
    "(\d+)", expand=False)
st_nd.loc[st_nd[st_dt_col].str.len() < 6, st_dt_col] = None
st_nd.loc[st_nd[end_dt_col].str.len() < 6, end_dt_col] = None

# backfills the missing dates with the previous date
st_nd[st_dt_col].bfill(inplace=True)
st_nd[end_dt_col].bfill(inplace=True)
st_nd[st_tme_col].bfill(inplace=True)
st_nd[end_tme_col].bfill(inplace=True)

# adds century to year if it is not there
st_nd[st_dt_col] = st_nd[st_dt_col].apply(
    lambda x: str(date.today())[:2] + x if len(x) == 6 and int(x[:2]) <= 50
    else str(int(str(date.today())[:2])-1) + x
    if len(x) == 6 and int(x[:2]) > 50
    else x)
st_nd[end_dt_col] = st_nd[end_dt_col].apply(
    lambda x: str(date.today())[:2] + x if len(x) == 6 and int(x[:2]) <= 50
    else str(int(str(date.today())[:2])-1) + x
    if len(x) == 6 and int(x[:2]) > 50
    else x)

# index 2 and 4 are time, this makes the time uniform length
st_nd[st_tme_col] = st_nd[st_tme_col].str.pad(
    width=7, side='right', fillchar="0")
st_nd[end_tme_col] = st_nd[end_tme_col].str.pad(
    width=7, side='right', fillchar="0")

# this filters for time = 24H00m and makes it zero hour
st_nd[end_tme_col].loc[st_nd[end_tme_col].str[:2]
                        == '24'] = ('0').zfill(7)

try:  # Looks for invalid date then changes to date format
    st_nd[st_dt_col] = st_nd[st_dt_col].apply(
        lambda x: pd.to_datetime(str(int(x)+1), format="%Y%m%d")
        if x[-2:] == "00"
        else pd.to_datetime(x, format="%Y%m%d").date())
except ValueError:
    traceback.print_exc()

try:  # adds a day if the hour is zero hour and changes string to datetime.date
    st_nd[end_dt_col] = st_nd[end_dt_col].apply(
        lambda x: pd.to_datetime(
            x[end_dt_col], format="%Y%m%d").date() + timedelta(days=1)
        if x[4] == '0'.zfill(7) else pd.to_datetime(x[end_dt_col], format="%Y%m%d").date())
except (ValueError, TypeError):
    # print(self.file)
    st_nd[end_dt_col] = pd.to_datetime(
        st_nd[end_dt_col], format="%Y%m%d").dt.date

# changes time string into datetime.time
try:
    st_nd[st_tme_col] = pd.to_datetime(
        st_nd[st_tme_col], format="%H%M%S%f").dt.time
except ValueError:
    st_nd[st_tme_col] = pd.to_datetime(
        "0".zfill(7), format="%H%M%S%f").strftime("%H:%M:%S")
try:
    st_nd[end_tme_col] = pd.to_datetime(
        st_nd[end_tme_col], format="%H%M%S%f").dt.time
except ValueError:
    st_nd[end_tme_col] = pd.to_datetime(
        "0".zfill(7), format="%H%M%S%f").strftime("%H:%M:%S")
except:
    pass
    # raise Exception(f"""header func: time not working 
    # - st_nd[st_tme_col] = st_nd[st_tme_col].apply(lambda x: pd.to_datetime(x, format='%H%M%S%f').time())
    #     look at file {self.file}""")

# creates start_datetime and end_datetime
st_nd["start_datetime"] = pd.to_datetime((st_nd[st_dt_col].astype(
    str)+st_nd[st_tme_col].astype(str)), format='%Y-%m-%d%H:%M:%S')
st_nd["end_datetime"] = pd.to_datetime((st_nd[end_dt_col].astype(
    str)+st_nd[end_tme_col].astype(str)), format='%Y-%m-%d%H:%M:%S')

st_nd = st_nd.iloc[:, 1:].drop_duplicates()

headers = pd.DataFrame()
headers['start_datetime'] = st_nd.groupby(
    st_nd['start_datetime'].dt.year).min()['start_datetime']
headers['end_datetime'] = st_nd.groupby(
    st_nd['end_datetime'].dt.year).max()['end_datetime']

# headers['document_url'] = self.file
# headers["header_id"] = self.header_id
headers['year'] = headers['start_datetime'].dt.year.round().astype(int)
headers["number_of_lanes"] = int(df.loc[df[0] == "L0", 2].drop_duplicates().reset_index(drop=True).at[0])

t21 = df.loc[df[0] == "21"].dropna(
    axis=1).drop_duplicates().reset_index().copy()
t21 = t21.iloc[:, (t21.shape[1])-9:].astype(int)
t21.columns = range(t21.columns.size)
t21.rename(columns={
    0: 'speedbin1',
    1: 'speedbin2',
    2: 'speedbin3',
    3: 'speedbin4',
    4: 'speedbin5',
    5: 'speedbin6',
    6: 'speedbin7',
    7: 'speedbin8',
    8: 'speedbin9'}, inplace=True)

headers = pd.concat([headers, t21], ignore_index=True,
                    axis=0).bfill().ffill().drop_duplicates()

try:
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "21", 1].drop_duplicates().reset_index(drop=True)[0])
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "21", 1].drop_duplicates().reset_index(drop=True)[0])
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "21", 1].drop_duplicates().reset_index(drop=True)[0])
except KeyError:
    pass
try:
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "30", 1].drop_duplicates().reset_index(drop=True)[0])
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "30", 1].drop_duplicates().reset_index(drop=True)[0])
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "30", 1].drop_duplicates().reset_index(drop=True)[0])
    headers["vehicle_classification_scheme"] = int(
        df.loc[df[0] == "30", 3].drop_duplicates().reset_index(drop=True)[0])
except KeyError:
    pass
try:
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "70", 1].drop_duplicates().reset_index(drop=True)[0])
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "70", 1].drop_duplicates().reset_index(drop=True)[0])
    headers["summary_interval_minutes"] = int(
        df.loc[df[0] == "70", 1].drop_duplicates().reset_index(drop=True)[0])
except KeyError:
    pass

try:
    headers['dir1_id'] = int(df[df[0] == "L1"].dropna(
        axis=1).drop_duplicates().reset_index(drop=True)[2].min())
    headers['dir2_id'] = int(df[df[0] == "L1"].dropna(
        axis=1).drop_duplicates().reset_index(drop=True)[2].max())
except (KeyError, ValueError):
    headers['dir1_id'] = 0
    headers['dir2_id'] = 4

try:
    headers["vehicle_classification_scheme"] = int(df.loc[
        df[0] == "70", 2].drop_duplicates().reset_index(drop=True)[0])
    headers["maximum_gap_milliseconds"] = int(df.loc[
        df[0] == "70", 3].drop_duplicates().reset_index(drop=True)[0])
    headers["maximum_differential_speed"] = int(df.loc[
        df[0] == "70", 4].drop_duplicates().reset_index(drop=True)[0])
    headers["error_bin_code"] = int(df.loc[
        df[0] == "70", 5].drop_duplicates().reset_index(drop=True)[0])
except KeyError:
    pass

headers = headers.reset_index(drop=True)

headers['year'] = headers['year'].astype(float).round().astype(int)

headers[headers.select_dtypes(include=['object','number']).columns] = headers[
    headers.select_dtypes(include=['object','number']).columns].apply(pd.to_numeric, errors='ignore')

station_name = df.loc[df[0].isin(["S0"]), 3:].reset_index(
    drop=True).drop_duplicates().dropna(axis=1)
headers["station_name"] = station_name[station_name.columns].apply(
    lambda row: ' '.join(row.values.astype(str)), axis=1)
    
# headers['site_id'] = self.site_id

# m = headers.select_dtypes(np.number)
# headers[m.columns] = m.round().astype(int)
# return headers



In [31]:
headers[headers.select_dtypes(include=[np.number]).columns] = headers[
    headers.select_dtypes(include=[np.number]).columns].apply(pd.to_numeric, axis=1, errors='ignore')

In [24]:
headers[headers.select_dtypes(include=[np.number]).columns] = headers[
    headers.select_dtypes(include=[np.number]).columns].astype(float).round().astype(int)

In [32]:
headers.dtypes

start_datetime                   datetime64[ns]
end_datetime                     datetime64[ns]
year                                    float64
number_of_lanes                         float64
speedbin1                               float64
speedbin2                               float64
speedbin3                               float64
speedbin4                               float64
speedbin5                               float64
speedbin6                               float64
speedbin7                               float64
speedbin8                               float64
speedbin9                               float64
summary_interval_minutes                float64
vehicle_classification_scheme           float64
dir1_id                                 float64
dir2_id                                 float64
station_name                             object
dtype: object

In [237]:
def process_datetimes(df):
    if df.loc[df[0] == "10"].empty:
        duration_min_col = 4
        date_col = 2
        time_col = 3
        add_day = 1
        date_col_name = "end_date"
        time_col_name = "end_time"
        typ = "sum"
        max_time_str_len = int(df[time_col].str.len().max())
    else:
        date_col = 4
        time_col = 5
        add_day = 0
        date_col_name = "departure_date"
        time_col_name = "departure_time"
        typ = "indv"
        max_time_str_len = int(df[time_col].str.len().max())
    date_fmt = "%Y%m%d"  # YYYYMMDD
    date_format = "%Y-%m-%d"  # YYYY-MM-DD
    time_fmt = "%H%M%S%f"  # (includes 00000000, 000000, 0000000)
    time_format = "%H:%M:%S.%f"  # 00:00:00.00

    df[time_col] = df[time_col].str.pad(
        width=max_time_str_len, side='right', fillchar="0")
    df[time_col].loc[df[time_col].str[:2] == '24'] = (
        '0').zfill(max_time_str_len)

    df[date_col] = df[date_col].apply(lambda x: str(date.today())[:2]+x if int(x[:2]) <= 50 and len(
        x) == 6 else (str(int(str(date.today())[:2])-1)+x if int(x[:2]) > 50 and len(x) == 6 else x))

    if typ == "indv":
        df[date_col] = pd.to_datetime(
            df[date_col], format=date_fmt).dt.strftime(date_format)
    else:
        df[date_col] = df[date_col].apply(lambda x: pd.to_datetime(
            x, format=date_fmt).date() + timedelta(days=add_day)
            if x[time_col] in ['0'.zfill(max_time_str_len), '24'.ljust(max_time_str_len, '0')]
            else pd.to_datetime(x, format=date_fmt).date())

    df['end_datetime'] = pd.to_datetime((df[date_col].astype(str)+df[time_col].astype(str)),
                                        format=date_format+time_fmt)

    if df.loc[df[0] == "10"].empty:
        df['start_datetime'] = df.apply(lambda x: pd.to_datetime(str(x[date_col])+str(
            x[time_col]), format=date_format+time_fmt) - timedelta(minutes=int(x[duration_min_col])), axis=1)
    else:
        df['start_datetime'] = pd.to_datetime(
            df[date_col]+df[time_col], format=date_format+time_fmt)

    df[time_col] = pd.to_datetime(
        df[time_col], format=time_fmt).dt.strftime(time_format)

    df.rename(columns={
        date_col: date_col_name,
        time_col: time_col_name
    }, inplace=True)
    return df

In [238]:
df = process_datetimes(df)

In [239]:
def get_lanes(df):
    if df.empty:
        pass
    else:
        lanes = df.loc[
            df[0] == "L1"
        ].dropna(axis=1).drop_duplicates().reset_index(drop=True).copy()
        if lanes.empty:
            lanes = df.loc[
            df[0] == "L0"
            ].dropna(axis=1).drop_duplicates().reset_index(drop=True).copy()
            max_lanes = lanes[1].astype(int).at[0]
        else:
            lanes = lanes.drop([17, 18, 19, 20, 21, 22, 23,
                                24, 25], axis=1, errors='ignore')
            if lanes.shape[1] == 5:
                lanes.rename(columns={
                    1: "lane_number",
                    2: "direction_code",
                    3: "lane_type_code",
                    4: "traffic_stream_number"
                }, inplace=True)
            elif lanes.shape[1] == 11:
                lanes.rename(columns={
                    1: "lane_number",
                    2: "direction_code",
                    3: "lane_type_code",
                    4: "traffic_stream_number",
                    5: "traffic_stream_lane_position",
                    6: "reverse_direction_lane_number",
                    7: "vehicle_code",
                    8: "time_code",
                    9: "length_code",
                    10: "speed_code"
                }, inplace=True)
            elif lanes.shape[1] == 17:
                lanes.rename(columns={
                    1: "lane_number",
                    2: "direction_code",
                    3: "lane_type_code",
                    4: "traffic_stream_number",
                    5: "traffic_stream_lane_position",
                    6: "reverse_direction_lane_number",
                    7: "vehicle_code",
                    8: "time_code",
                    9: "length_code",
                    10: "speed_code",
                    11: "occupancy_time_code",
                    12: "vehicle_following_code",
                    13: "trailer_code",
                    14: "axle_code",
                    15: "mass_code",
                    16: "tyre_type_code"
                }, inplace=True)
            else:
                lanes.rename(columns={
                    1: "lane_number",
                    2: "direction_code",
                    3: "lane_type_code",
                    4: "traffic_stream_number"
                }, inplace=True)
            lanes[lanes.select_dtypes(include=['object']).columns] = lanes[
                lanes.select_dtypes(include=['object']).columns].apply(
                    pd.to_numeric, axis=1, errors='ignore')
            lanes["site_name"] = "site_id"
            try:
                max_lanes = int(lanes['lane_number'].astype(int).max())
            except ValueError:
                max_lanes = int(
                    lanes['lane_number'].drop_duplicates().count())
        return lanes, max_lanes

In [240]:
lanes, max_lanes = get_lanes(head_df)

In [267]:
def get_direction(df, lanes, max_lanes):
    try:
        if df.loc[df[0] == "10"].empty:
            lane_col = 5
        else:
            lane_col = 6
        
        try:
            dir_1 = lanes["direction_code"].astype(int).min()
            dir_2 = lanes["direction_code"].astype(int).max()
        except (TypeError, ValueError):
            dir_1 = 0
            dir_2 = 4

        direction_1 = 'P'
        direction_2 = 'N'

        if dir_1 == dir_2:
            dir_2 = dir_1
            direction_2 = direction_1
        else:
            pass

        df['direction'] = df[lane_col].astype(int)
        df['compass_heading'] = df[lane_col].astype(int)
        df['direction_code'] = df[lane_col].astype(int)
        df['direction'].loc[df[lane_col].astype(int).isin(
            list(lanes['lane_number'].astype(
                int).loc[lanes['direction_code'].astype(int) == dir_1])
        )] = direction_1
        df['direction'].loc[df[lane_col].astype(int).isin(
            list(lanes['lane_number'].astype(
                int).loc[lanes['direction_code'].astype(int) == dir_2])
        )] = direction_2
        df['compass_heading'].loc[df[lane_col].astype(int).isin(
            list(lanes['lane_number'].astype(
                int).loc[lanes['direction_code'].astype(int) == dir_1])
        )] = str(dir_1)
        df['compass_heading'].loc[df[lane_col].astype(int).isin(
            list(lanes['lane_number'].astype(
                int).loc[lanes['direction_code'].astype(int) == dir_2])
        )] = str(dir_2)
        df['direction_code'].loc[df[lane_col].astype(int).isin(
            list(lanes['lane_number'].astype(
                int).loc[lanes['direction_code'].astype(int) == dir_2])
        )] = str(dir_2)
        df['direction_code'].loc[df[lane_col].astype(int).isin(
            list(lanes['lane_number'].astype(
                int).loc[lanes['direction_code'].astype(int) == dir_1])
        )] = str(dir_1)

        return df
    except KeyError:
        df[lane_col] = df[lane_col].astype(int)
        df['direction'] = df.apply(
            lambda x: 'P' if int(x[lane_col]) <= int(max_lanes)/2 else 'N', axis=1)
        df['direction_code'] = df.apply(
            lambda x: 0 if int(x[lane_col]) <= int(max_lanes)/2 else 4, axis=1)
        df['compass_heading'] = df.apply(
            lambda x: 0 if int(x[lane_col]) <= int(max_lanes)/2 else 4, axis=1)
        return df

In [268]:
df = get_direction(df, lanes, max_lanes)

In [269]:
df.head()

,0,1,end_date,end_time,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,end_datetime,start_datetime,direction,direction_code,compass_heading
0,13,0,2010-01-01,00:03:50.200000,04,4,0,105,904,999,0,0,0,0,5,00,2,28,539,35,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2010-01-01 00:03:50.200,2009-12-31 23:59:50.200,N,4,4
1,13,0,2010-01-01,00:17:39.970000,04,4,0,73,1546,999,0,0,0,0,9,00,4,55,335,72,734,41,129,41,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2010-01-01 00:17:39.970,2010-01-01 00:13:39.970,N,4,4
2,13,0,2010-01-01,00:26:13.000000,04,4,0,80,1991,999,0,0,0,0,0,00,7,49,312,32,132,39,497,19,124,30,284,32,435,20,None,None,None,None,None,None,None,None,None,None,2010-01-01 00:26:13.000,2010-01-01 00:22:13.000,N,4,4
3,21,0,2010-01-01,01:00:00.000000,60,1,0001,0004,0019,0026,0043,0042,0021,0010,0006,0003,000187,001,000,001,0010,0010,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2010-01-01 01:00:00.000,2010-01-01 00:00:00.000,P,0,0
4,21,0,2010-01-01,01:00:00.000000,60,2,0000,0000,0000,0004,0009,0013,0021,0009,0008,0003,000000,000,000,000,0000,0000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2010-01-01 01:00:00.000,2010-01-01 00:00:00.000,P,0,0


In [270]:
# def type_30(self) -> pd.DataFrame:
data_df = df
if data_df is None:
    pass
else:
    data = data_df.loc[(data_df[0] == "30")].dropna(
        axis=1, how="all"
    ).reset_index(drop=True).copy()
    header = head_df.loc[(head_df[0] == "30")].dropna(
        axis=1, how="all"
    ).reset_index(drop=True).copy()


In [279]:
if data.empty:
    pass
else:
    if header.shape[1] > 3:
        classification_scheme = header.iloc[0, 3]
        number_of_data_records = header.iloc[0, 4]
    else:
        classification_scheme = header.iloc[0, 2]
        number_of_data_records = header.iloc[0, 3]

    # vc_df = select_classification_scheme(classification_scheme)

    if data[1].isin(["0", "2", 0, 2]).any():
        ddf = data.iloc[:, 1:].dropna(
            axis=1, how="all").reset_index(drop=True)

        ddf[ddf.select_dtypes(include=['object']).columns] = ddf[
            ddf.select_dtypes(include=['object']).columns].apply(
            pd.to_numeric, axis=1, errors='ignore')

        ddf['vehicle_classification_scheme'] = int(
            classification_scheme)

        ddf.columns = ddf.columns.astype(str)

        df3 = pd.DataFrame(columns=[
            'edit_code',
            'start_datetime',
            'end_date',
            'end_time',
            'duration_of_summary',
            'lane_number',
            'number_of_vehicles',
            'class_number',
            'direction',
            'compass_heading'])
        for lane_no in range(1, int(max_lanes)+1):
            for i in range(6, int(number_of_data_records)+6):
                join_to_df3 = ddf.loc[ddf['5'].astype(int) == lane_no, [
                    '1', 'start_datetime', 'end_date', 'end_time', '4', '5', str(i), 'direction', 'compass_heading']]
                join_to_df3['class_number'] = i-5
                join_to_df3.rename(columns={
                    '1': "edit_code",
                    '2': "end_date",
                    '3': "end_time",
                    '4': "duration_of_summary",
                    '5': 'lane_number',
                    str(i): 'number_of_vehicles'
                }, inplace=True)
                # df3 = pd.concat([df3,join_to_df3],keys=['start_datetime','lane_number','number_of_vehicles','class_number'],ignore_index=True, axis=0)
                df3 = pd.concat([df3, join_to_df3], keys=[
                                'start_datetime', 'lane_number'], ignore_index=True, axis=0)
        df3 = df3.apply(pd.to_numeric, axis=1, errors="ignore")
        df3['classification_scheme'] = int(classification_scheme)
        df3['site_id'] = 'self.site_id'
        df3["header_id"] = 'self.header_id'
        df3['year'] = int(data['start_datetime'].at[0].year)
    else:
        pass
    # return df3

In [280]:
df3[['edit_code','duration_of_summary','lane_number','number_of_vehicles','class_number','compass_heading','classification_scheme','year']] = df3[['edit_code','duration_of_summary','lane_number','number_of_vehicles','class_number','compass_heading','classification_scheme','year']].apply(pd.to_numeric, errors="ignore")

In [282]:
df3.head()

,edit_code,start_datetime,end_date,end_time,duration_of_summary,lane_number,number_of_vehicles,class_number,direction,compass_heading,classification_scheme,site_id,header_id,year
0,0,2010-01-01 00:00:00,2010-01-01,01:00:00.000000,60,1,4,1,P,0,8,self.site_id,self.header_id,2010
1,0,2010-01-01 01:00:00,2010-01-01,02:00:00.000000,60,1,8,1,P,0,8,self.site_id,self.header_id,2010
2,0,2010-01-01 02:00:00,2010-01-01,03:00:00.000000,60,1,6,1,P,0,8,self.site_id,self.header_id,2010
3,0,2010-01-01 03:00:00,2010-01-01,04:00:00.000000,60,1,2,1,P,0,8,self.site_id,self.header_id,2010
4,0,2010-01-01 04:00:00,2010-01-01,05:00:00.000000,60,1,8,1,P,0,8,self.site_id,self.header_id,2010


In [62]:
t21_data = type_21(df)

In [64]:
def header_calcs(header: pd.DataFrame, data: pd.DataFrame, type: int) -> pd.DataFrame:
    try:
        speed_limit_qry = f"select max_speed from trafc.countstation where tcname = '{data['site_id'].iloc[0]}' ;"
        speed_limit = pd.read_sql_query(speed_limit_qry,config.ENGINE).reset_index(drop=True)
        try:
            speed_limit = speed_limit['max_speed'].iloc[0]
        except IndexError:
            speed_limit = 60
        data = data.fillna(0, axis=0)
        if type == 21:
            try:
                header['adt_total'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D')]).sum().mean().round().astype(int)
                header['adt_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P']]).sum().mean())
                header['adt_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N']]).sum().mean())

                header['adtt_total'] = data['total_heavy_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D')]).sum().mean().round().astype(int)
                header['adtt_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P']]).sum().mean())
                header['adtt_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N']]).sum().mean())

                header['total_vehicles'] = data['total_vehicles_type21'].sum()
                header['total_vehicles_positive_direction'] = data['total_vehicles_type21'].groupby(data['direction'].loc[data['direction'] == 'P']).sum()[0]
                header['total_vehicles_negative_direction'] = data['total_vehicles_type21'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[0]

                header['total_heavy_vehicles'] = data['total_heavy_vehicles_type21'].sum()
                header['total_heavy_negative_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['truck_split_negative_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / data['total_heavy_vehicles_type21'].sum()
                header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / data['total_heavy_vehicles_type21'].sum()

                header['total_light_vehicles'] = data['total_light_vehicles_type21'].sum()
                header['total_light_positive_direction'] = data['total_light_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['total_light_negative_direction'] = data['total_light_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]

                header['short_heavy_vehicles'] = data['short_heavy_vehicles'].sum()
                header['short_heavy_positive_direction'] = data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['short_heavy_negative_direction'] = data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]

                header['Medium_heavy_vehicles'] = data['medium_heavy_vehicles'].sum()
                header['Medium_heavy_negative_direction'] = data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                header['Medium_heavy_positive_direction'] = data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]

                header['long_heavy_vehicles'] = data['long_heavy_vehicles'].sum()
                header['long_heavy_positive_direction'] = data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['long_heavy_negative_direction'] = data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]

                header['vehicles_with_rear_to_rear_headway_less_than_2sec_positive_dire'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                header['vehicles_with_rear_to_rear_headway_less_than_2sec_negative_dire'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                header['vehicles_with_rear_to_rear_headway_less_than_2sec_total'] = data['rear_to_rear_headway_shorter_than_2_seconds'].sum()
            
                header['type_21_count_interval_minutes'] = data['duration_min'].mean()

                header['highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P']]).sum().max())
                header['highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N']]).sum().max())
                header['highest_volume_per_hour_total'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H')]).sum().max()

                header['15th_highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P']]).sum().quantile(q=0.15,  interpolation='linear'))
                header['15th_highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N']]).sum().quantile(q=0.15,  interpolation='linear'))
                header['15th_highest_volume_per_hour_total'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H')]).sum().quantile(q=0.15, interpolation='linear'))
                
                header['30th_highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'P']]).sum().quantile(q=0.3,  interpolation='linear'))
                header['30th_highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N']]).sum().quantile(q=0.3, interpolation='linear'))
                header['30th_highest_volume_per_hour_total'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H')]).sum().quantile(q=0.3, interpolation='linear'))

                # header['average_speed_positive_direction'] = 
                # header['average_speed_negative_direction'] = 
                try:
                    header['average_speed'] = (int((
                    (header['speedbin1'] * data['speedbin1'].sum()) +
                    (header['speedbin2'] * data['speedbin2'].sum()) +
                    (header['speedbin3'] * data['speedbin3'].sum()) +
                    (header['speedbin4'] * data['speedbin4'].sum()) +
                    (header['speedbin5'] * data['speedbin5'].sum()) +
                    (header['speedbin6'] * data['speedbin6'].sum()) +
                    (header['speedbin7'] * data['speedbin7'].sum()) +
                    (header['speedbin8'] * data['speedbin8'].sum()) +
                    (header['speedbin9'] * data['speedbin9'].sum()) 
                    ))
                    / data['sum_of_heavy_vehicle_speeds'].astype(int).sum()
                    )
                except TypeError:
                    header['average_speed'] = (((
                    (header['speedbin1'] * data['speedbin1'].astype(int).sum()) +
                    (header['speedbin2'] * data['speedbin2'].astype(int).sum()) +
                    (header['speedbin3'] * data['speedbin3'].astype(int).sum()) +
                    (header['speedbin4'] * data['speedbin4'].astype(int).sum()) +
                    (header['speedbin5'] * data['speedbin5'].astype(int).sum()) +
                    (header['speedbin6'] * data['speedbin6'].astype(int).sum()) +
                    (header['speedbin7'] * data['speedbin7'].astype(int).sum()) +
                    (header['speedbin8'] * data['speedbin8'].astype(int).sum()) +
                    (header['speedbin9'] * data['speedbin9'].astype(int).sum()) 
                    ))
                    / data['sum_of_heavy_vehicle_speeds'].astype(int).sum()
                    )
                # header['average_speed_light_vehicles_positive_direction'] = 
                # header['average_speed_light_vehicles_negative_direction'] = 
                header['average_speed_light_vehicles'] = ((
                    (header['speedbin1'] * data['speedbin1'].sum()) +
                    (header['speedbin2'] * data['speedbin2'].sum()) +
                    (header['speedbin3'] * data['speedbin3'].sum()) +
                    (header['speedbin4'] * data['speedbin4'].sum()) +
                    (header['speedbin5'] * data['speedbin5'].sum()) +
                    (header['speedbin6'] * data['speedbin6'].sum()) +
                    (header['speedbin7'] * data['speedbin7'].sum()) +
                    (header['speedbin8'] * data['speedbin8'].sum()) +
                    (header['speedbin9'] * data['speedbin9'].sum()) -
                    data['sum_of_heavy_vehicle_speeds'].sum()
                    )
                    / data['sum_of_heavy_vehicle_speeds'].sum()
                    )
                
                # header['average_speed_heavy_vehicles_positive_direction'] = 
                # header['average_speed_heavy_vehicles_negative_direction'] = 
                # header['average_speed_heavy_vehicles'] = 
                
                try:
                    header['truck_split_positive_direction'] = (str(
                        round(data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]*100)) 
                    + ' : ' +
                    str(
                        round(data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]*100)) 
                    + ' : ' +
                    str(
                        round(data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]*100))
                    )
                except ValueError:
                    pass
                
                try:
                    header['truck_split_negative_direction'] = (str(
                        round(data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()*100)) 
                        + ' : ' +
                    str(
                        round(data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]*100)) 
                    + ' : ' +
                    str(
                        round(data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / 
                        data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]*100))
                    )
                except ValueError:
                    pass
                try:
                    header['truck_split_total'] = (str(
                        round(data['short_heavy_vehicles'].sum() / 
                        data['total_heavy_vehicles_type21'].sum()*100)) 
                        + ' : ' +
                    str(
                        round(data['medium_heavy_vehicles'].sum() / 
                        data['total_heavy_vehicles_type21'].sum()*100)) 
                    + ' : ' +
                    str(
                        round(data['long_heavy_vehicles'].sum() / 
                        data['total_heavy_vehicles_type21'].sum()*100))
                    )
                except ValueError:
                    pass

            except KeyError:
                pass
            try:
                header["type_21_count_interval_minutes"] = header["type_21_count_interval_minutes"].round().astype(int)
            except (KeyError, pd.errors.IntCastingNaNError):
                pass

            return header
        
        elif type == 30:
            try:
                if header['adt_total'].isnull().all():
                    header['adt_total'] = data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D')]).sum().mean().astype(int)
                    header['adt_positive_direction'] = round(data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P']]).sum().mean())
                    header['adt_negative_direction'] = round(data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N']]).sum().mean())
                else:
                    pass

                if header['adtt_total'].isnull().all():
                    header['adtt_total'] = data['total_heavy_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D')]).sum().mean().astype(int)
                    header['adtt_positive_direction'] = round(data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'P']]).sum().mean())
                    header['adtt_negative_direction'] = round(data['total_vehicles_type30'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N']]).sum().mean())
                else:
                    pass

                if header['total_vehicles'].isnull().all():
                    header['total_vehicles'] = data['total_vehicles_type30'].sum()
                    header['total_vehicles_positive_direction'] = data['total_vehicles_type30'].groupby(data['direction'].loc[data['direction'] == 'P']).sum()[0]
                    header['total_vehicles_negative_direction'] = data['total_vehicles_type30'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[0]
                else:
                    pass
                
                if header['total_heavy_vehicles'].isnull().all():
                    header['total_heavy_vehicles'] = data['total_heavy_vehicles_type21'].sum()
                    header['total_heavy_negative_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                    header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                    header['truck_split_negative_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0] / data['total_heavy_vehicles_type21'].sum()
                    header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0] / data['total_heavy_vehicles_type21'].sum()
                else:
                    pass

                if header['total_light_vehicles'].isnull().all():
                    header['total_light_vehicles'] = data['total_light_vehicles_type30'].sum()
                    header['total_light_positive_direction'] = data['total_light_vehicles_type30'].groupby([data['direction'].loc[data['direction'] == 'P']]).sum()[0]
                    header['total_light_negative_direction'] = data['total_light_vehicles_type30'].groupby([data['direction'].loc[data['direction'] == 'N']]).sum()[0]
                else:
                    pass
            except KeyError:
                pass
                
            try:
                header['type_30_vehicle_classification_scheme'] = header['type_30_vehicle_classification_scheme'].round().astype(int)
            except (KeyError, pd.errors.IntCastingNaNError):
                pass

            return header

        elif type == 70:

            try:
                header['type_70_maximum_gap_milliseconds'] = header['type_70_maximum_gap_milliseconds'].round().astype(int)
            except (KeyError, pd.errors.IntCastingNaNError):
                pass
            
            return header
        
        elif type == 10:
            header['total_light_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_light_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_light_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']<=1].count()[0].round().astype(int)
            header['total_heavy_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_heavy_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_heavy_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']>1].count()[0].round().astype(int)
            header['total_short_heavy_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_short_heavy_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_short_heavy_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0].round().astype(int)
            header['total_medium_heavy_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_medium_heavy_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_medium_heavy_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0].round().astype(int)
            header['total_long_heavy_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0].round().astype(int)
            header['total_long_heavy_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0].round().astype(int)
            header['total_long_heavy_vehicles'] = data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0].round().astype(int)
            header['total_vehicles_negative_direction'] = data.loc[data['direction']=='N'].count()[0].round().astype(int)
            header['total_vehicles_positive_direction'] = data.loc[data['direction']=='P'].count()[0].round().astype(int)
            header['total_vehicles'] = data.count()[0].round().astype(int)
            header['average_speed_negative_direction'] = data.loc[data['direction']=='N']['vehicle_speed'].mean().round(2)
            header['average_speed_positive_direction'] = data.loc[data['direction']=='P']['vehicle_speed'].mean().round(2)
            header['average_speed'] = data['vehicle_speed'].mean().round(2)
            header['average_speed_light_vehicles_negative_direction'] = data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='N')].mean().round(2)
            header['average_speed_light_vehicles_positive_direction'] = data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='P')].mean().round(2)
            header['average_speed_light_vehicles'] = data['vehicle_speed'].loc[data['vehicle_class_code_secondary_scheme']<=1].mean().round(2)
            header['average_speed_heavy_vehicles_negative_direction'] = data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].mean().round(2)
            header['average_speed_heavy_vehicles_positive_direction'] = data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].mean().round(2)
            header['average_speed_heavy_vehicles'] = data['vehicle_speed'].loc[data['vehicle_class_code_secondary_scheme']>1].mean().round(2)
            header['truck_split_negative_direction'] = {str((((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int))}
            header['truck_split_positive_direction'] = {str((((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int))}
            header['truck_split_total'] = {str((((data.loc[data['vehicle_class_code_secondary_scheme']==2].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[data['vehicle_class_code_secondary_scheme']==3].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[data['vehicle_class_code_secondary_scheme']==4].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int))}
            header['estimated_axles_per_truck_negative_direction'] = ((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]*2+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]*5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0]*7)/(data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0])).round(2)
            header['estimated_axles_per_truck_positive_direction'] = ((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]*2+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]*5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0]*7)/(data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0])).round(2)
            header['estimated_axles_per_truck_total'] = ((data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]*2+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]*5+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0]*7)/(data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0])).round(2)
            header['percentage_speeding_positive_direction'] = ((data.loc[(data['vehicle_speed']>speed_limit)&(data['direction']=='P')].count()[0]/data.loc[data['direction'=='P']].count()[0])*100).round(2)
            header['percentage_speeding_negative_direction'] = ((data.loc[(data['vehicle_speed']>speed_limit)&(data['direction']=='N')].count()[0]/data.loc[data['direction'=='N']].count()[0])*100).round(2)
            header['percentage_speeding_total'] = ((data.loc[data['vehicle_speed']>speed_limit].count()[0]/data.count()[0])*100).round(2)
            header['vehicles_with_rear_to_rear_headway_less_than_2sec_negative_dire'] = data.loc[(data['vehicle_following_code']==2)&data['direction']=='N'].count()[0]
            header['vehicles_with_rear_to_rear_headway_less_than_2sec_positive_dire'] = data.loc[(data['vehicle_following_code']==2)&data['direction']=='P'].count()[0]
            header['vehicles_with_rear_to_rear_headway_less_than_2sec_total'] = data.loc[data['vehicle_following_code']==2].count()[0]
            header['estimated_e80_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]*0.6+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]*2.5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0]*2.1
            header['estimated_e80_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]*0.6+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]*2.5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0]*2.1
            header['estimated_e80_on_road'] = data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]*0.6+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]*2.5+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0]*2.1
            header['adt_negative_direction'] = data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adt_positive_direction'] = data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adt_total'] = data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adtt_negative_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adtt_positive_direction'] = data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['adtt_total'] = data.loc[data['vehicle_class_code_secondary_scheme']>1].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)
            header['highest_volume_per_hour_negative_direction'] = data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]
            header['highest_volume_per_hour_positive_direction'] = data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]
            header['highest_volume_per_hour_total'] = data.groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]
            header["15th_highest_volume_per_hour_negative_direction"] = round(data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].round().astype(int))
            header["15th_highest_volume_per_hour_positive_direction"] = round(data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].round().astype(int))
            header["15th_highest_volume_per_hour_total"] = data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].round().astype(int)
            header["30th_highest_volume_per_hour_negative_direction"] = round(data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].round().astype(int))
            header["30th_highest_volume_per_hour_positive_direction"] = round(data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].round().astype(int))
            header["30th_highest_volume_per_hour_total"] = data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].round().astype(int)
            header["15th_percentile_speed_negative_direction"] = data.loc[data['direction']=='N']['vehicle_speed'].quantile(0.15).round(2)
            header["15th_percentile_speed_positive_direction"] = data.loc[data['direction']=='P']['vehicle_speed'].quantile(0.15).round(2)
            header["15th_percentile_speed_total"] = data['vehicle_speed'].quantile(0.15).round(2)
            header["85th_percentile_speed_negative_direction"] = data.loc[data['direction']=='N']['vehicle_speed'].quantile(0.85).round(2)
            header["85th_percentile_speed_positive_direction"] = data.loc[data['direction']=='P']['vehicle_speed'].quantile(0.85).round(2)
            header["85th_percentile_speed_total"] = data['vehicle_speed'].quantile(0.85).round(2)
            header['avg_weekday_traffic'] = data.groupby(pd.Grouper(key='start_datetime',freq='B')).count().mean()[0].round().astype(int)
            header['number_of_days_counted'] = data.groupby([data['start_datetime'].dt.to_period('D')]).count().count()[0]
            header['duration_hours'] = data.groupby([data['start_datetime'].dt.to_period('H')]).count().count()[0]

            return header

        elif type == 60:
            
            return header
        else:
            return header
    except IndexError:
        return header



In [2]:
if df is None:
    pass
else:
    # data = self.data_df
    data = df.loc[(df[0] == "10")].dropna(
        axis=1, how="all"
    ).reset_index(drop=True).copy()

    num_of_fields = int(data.iloc[:,1].unique()[0])
    ddf = data.iloc[:,: 2 + num_of_fields]


    ddf.rename(columns=config.RENAME_TYPE10_DATA_COLUMNS, inplace=True, errors='ignore')

    ddf["data_id"] = ddf.apply(lambda x: uuid.uuid4(), axis=1)

NameError: name 'df' is not defined

In [16]:
ddf.head()

,data_type_code,number_of_fields_associated_with_the_basic_vehicle_data,data_source_code,edit_code,departure_date,departure_time,assigned_lane_number,physical_lane_number,forward_reverse_code,vehicle_category,vehicle_class_code_primary_scheme,vehicle_class_code_secondary_scheme,vehicle_speed,vehicle_length,site_occupancy_time_in_milliseconds,chassis_height_code,vehicle_following_code,data_id
0,10,15,1,0,2022-02-10,15:15:00.000000,2,2,1,12,2,1,116,590,,3,1,d6d2383b-b26e-4118-9acf-16c7e747d510
1,10,15,1,0,2022-02-10,15:15:00.800000,3,3,1,12,2,1,100,530,,1,2,419b8fdf-e5a1-4b98-8019-f5627cb35d97
2,10,15,1,0,2022-02-10,15:15:01.500000,2,2,1,12,2,1,107,500,,3,2,01e9545d-d745-4aa9-95a8-b2f2e03b41a4
3,10,18,1,0,2022-02-10,15:15:01.700000,1,1,1,12,2,1,69,460,,3,2,fcade08d-c1cb-4b71-98dd-58710d447a2e
4,10,18,1,0,2022-02-10,15:15:02.200000,4,4,1,12,2,1,105,490,,3,2,91a244ff-0eec-4062-a06a-927055f0b5e3


In [24]:
int(head_df.loc[head_df[0]=="10", 1].values[0])

8

In [50]:
if data.shape[1] > ddf.shape[1]:
    sub_data_df = pd.DataFrame(columns=['sub_data_type_code','offset_sensor_detection_code','mass_measurement_resolution_kg', 'number','value'])
    for index, row in data.iterrows():
        col = int(row[1]) + 2
        while col < len(row) - 5 and row[col] != None:
            sub_data_type = row[col]
            col += 1
            no_of_type = int(row[col])
            col += 1
            if sub_data_type[0].lower() in ['w','a','g']:
                odc = row[col]
                col += 1
                mmr = row[col]
                col +=1
                for i in range(0,no_of_type):
                    tempdf = pd.DataFrame([[
                    sub_data_type,
                    odc,
                    mmr,
                    i + 1,
                    row[col]]
                    ], columns = [
                    'sub_data_type_code',
                    'offset_sensor_detection_code',
                    'mass_measurement_resolution_kg',
                    'number',
                    'value'
                    ])
                    sub_data_df = pd.concat([sub_data_df, tempdf])
                    col += 1
            elif sub_data_type[0].lower() in ['s','t','c']:
                for i in range(0,no_of_type):
                    tempdf = pd.DataFrame([[ 
                    sub_data_type,
                    i + 1,
                    row[col]]], columns = [
                    'sub_data_type_code',
                    'number',
                    'value'])
                    sub_data_df = pd.concat([sub_data_df, tempdf])
                    col += 1
            elif sub_data_type[0].lower() in ['v']:
                odc = row[col]
                col += 1
                for i in range(0,no_of_type):
                    tempdf = pd.DataFrame([[
                    sub_data_type,
                    odc,
                    i + 1,
                    row[col]]
                    ], columns = [
                    'sub_data_type_code',
                    'offset_sensor_detection_code',
                    'number',
                    'value'
                    ])
                    sub_data_df = pd.concat([sub_data_df, tempdf])
                    col += 1
else:
    sub_data_df = pd.DataFrame(columns=['index','sub_data_type_code','offset_sensor_detection_code','mass_measurement_resolution_kg', 'number','value'])

    sub_data_df = sub_data_df.merge(ddf['data_id'], how='left', left_index=True, right_index=True)
    
    ddf = ddf.fillna(0)
    ddf["assigned_lane_number"] = ddf["assigned_lane_number"].astype(int)
    ddf["lane_number"] = ddf["physical_lane_number"].astype(int)
    ddf["physical_lane_number"] = ddf["physical_lane_number"].astype(int)

    ddf = ddf.replace(r'^\s*$', np.NaN, regex=True)
    sub_data_df = sub_data_df.replace(r'^\s*$', np.NaN, regex=True)
    sub_data_df = sub_data_df.drop("index", axis=1)

    scols = ddf.select_dtypes('object').columns
    ddf[scols] = ddf[scols].apply(pd.to_numeric, axis=1, errors='ignore')

    ddf['year'] = ddf['start_datetime'].dt.year
    ddf["site_id"] = "site_id"
    ddf["header_id"] = 'header_id'

    ddf = ddf[ddf.columns.intersection(t10_cols)]

1
2
0
1
1
2
2
3
1
2
5
6
1
2
1
2
0
0
1
2
0
1
0
1
1
2
1
2
0
0
1
2
0
1
1
2
1
2
1
2
0
0
1
2
1
2
0
0
0
1
0
0
1
2
0
1
1
2
1
2
0
1
1
2
0
1
1
2
1
2
0
1
0
0
0
1
1
2
1
2
2
3
0
0
1
2
0
1
0
0
0
0
0
1
0
1
1
2
1
2
1
2
5
6
0
0
3
4
0
0
0
1
0
0
0
1
0
1
2
3
0
1
0
1
1
2
0
0
0
1
0
0
0
1
1
2
1
2
0
0
1
2
1
2
1
2
1
2
1
2
0
1
1
2
1
2
0
1
0
1
0
1
0
1
1
2
1
2
0
1
1
2
0
1
0
1
1
2
1
2
0
1
0
1
0
0
0
0
0
0
0
1
1
2
1
2
0
1
0
0
0
1
1
2
1
2
1
2
0
1
1
2
0
1
1
2
1
2
1
2
0
0
0
1
1
2
1
2
1
2
1
2
0
0
0
1
1
2
0
0
0
0
0
1
1
2
1
2
0
0
1
2
0
0
0
1
0
1
1
2
1
2
0
1
1
2
1
2
0
0
1
2
0
0
1
2
1
2
1
2
0
0
1
2
1
2
0
0
0
1
1
2
0
1
0
1
1
2
0
1
0
1
0
0
1
2
1
2
1
2
1
2
1
2
0
1
1
2
1
2
0
1
0
1
0
0
0
1
0
1
0
1
1
2
0
0
2
3
0
1
1
2
0
1
0
0
0
1
0
1
0
0
0
1
0
1
0
1
1
2
0
1
1
2
0
1
1
2
1
2
0
1
0
1
1
2
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
1
2
1
2
1
2
1
2
1
2
0
0
0
0
0
0
1
2
0
0
1
2
1
2
0
0
0
0
1
2
0
0
0
0
1
2
0
0
1
2
1
2
0
0
0
1
0
0
1
2
1
2
0
0
0
0
1
2
0
0
0
1
1
2
0
1
0
0
6
7
6
7
1
2
1
2
0
1
1
2
1
2
1
2
1
2
1
2
1
2
0
0
1
2
0
1
1
2
1
2
0
1
1
2


In [1]:
print(sub_data_df.head())

NameError: name 'sub_data_df' is not defined

In [19]:
pd.concat

,data_type_code,number_of_fields_associated_with_the_basic_vehicle_data,data_source_code,edit_code,departure_date,departure_time,assigned_lane_number,physical_lane_number,forward_reverse_code,vehicle_category,vehicle_class_code_primary_scheme,vehicle_class_code_secondary_scheme,vehicle_speed,vehicle_length,site_occupancy_time_in_milliseconds,chassis_height_code,vehicle_following_code,data_id
0,10,15,1,0,211201,0000006,3,3,1,10,1,1,82,437,233,3,1,503cf23c-4845-45a5-b7c8-5391279c579b
1,10,15,1,0,211201,0000128,2,2,1,10,1,1,85,406,215,3,1,632da10d-9136-4b37-bbe9-e4e4ff66e0cb
2,10,15,1,0,211201,0000221,4,4,1,10,1,1,72,546,296,1,1,6cf05322-f1b3-4cb0-b5f0-cabb5d07bd6b
3,10,15,1,0,211201,0000291,1,1,1,10,1,1,69,442,278,2,1,ae0a03bc-5ad2-473b-83b4-ca5f7bd01b02
4,10,15,1,0,211201,0000472,2,2,1,10,1,1,73,456,271,3,1,2147ad39-6bb5-4321-918a-89152faee3f4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,10,15,1,0,211201,0450384,1,1,1,10,1,1,55,453,328,1,1,19605f8c-efb0-4802-9198-6b4dfa84cd53
483,10,15,1,0,211201,0451329,1,1,1,10,1,1,90,483,235,3,1,5ff2a85d-15e3-4af9-8069-02d739142a80
484,10,15,1,0,211201,0451580,4,4,1,10,1,1,82,371,206,3,1,12566212-5cbf-4d72-9353-607f4ba8376e
485,10,15,1,0,211201,0452027,4,4,1,10,1,1,90,451,220,3,1,149c516f-1fb0-44c1-91b6-b3f6e31c1da6
